# Facial Mood Detection using openCV


In [1]:
!pip uninstall -y opencv-python opencv-contrib-python
!pip install opencv-contrib-python


Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (73.2 MB)


In [2]:
# Core libraries
import cv2
import numpy as np
import os
from pathlib import Path

print("OpenCV version:", cv2.__version__)


OpenCV version: 4.12.0


In [3]:
emotion_map = {
    0: "anger",
    1: "contempt",
    2: "disgust",
    3: "fear",
    4: "happy",
    5: "sadness",
    6: "surprise"
}

emotion_map_inv = {v: k for k, v in emotion_map.items()}


In [4]:
#Load haarcascade for face detection
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

if face_cascade.empty():
    raise IOError("Haar Cascade not loaded")


In [5]:
#Face detection and preprocessing
def extract_face(img_path, target_size=(200, 200)):
    img = cv2.imread(str(img_path))

    if img is None:
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5
    )

    if len(faces) == 0:
        return None

    # Take largest detected face
    x, y, w, h = sorted(
        faces, key=lambda f: f[2]*f[3], reverse=True
    )[0]

    face_roi = gray[y:y+h, x:x+w]
    face_resized = cv2.resize(face_roi, target_size)

    return face_resized


In [6]:
!unzip "archive (6).zip" -d /content/ck_plus_dataset


Archive:  archive (6).zip
replace /content/ck_plus_dataset/CK+48/anger/S010_004_00000017.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
from pathlib import Path

data_dir = Path("/content/ck_plus_dataset/CK+48")
assert data_dir.exists(), "Dataset path not found"

faces = []
labels = []

for emotion_folder in sorted(data_dir.iterdir()):
    if not emotion_folder.is_dir():
        continue

    emotion_label = emotion_map_inv.get(emotion_folder.name)
    if emotion_label is None:
        continue

    for img_path in emotion_folder.glob("*.png"):
        face = extract_face(img_path)
        if face is not None:
            faces.append(face)
            labels.append(emotion_label)

print("Total samples collected:", len(faces))


Total samples collected: 2


In [8]:
import numpy as np

faces = np.array(faces, dtype="uint8")
labels = np.array(labels)

print(type(faces), type(labels))
print(faces.shape, labels.shape)


<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2, 200, 200) (2,)


In [9]:
indices = np.arange(len(faces))
np.random.shuffle(indices)

split = int(0.8 * len(indices))

train_idx = indices[:split]
test_idx  = indices[split:]

X_train, y_train = faces[train_idx], labels[train_idx]
X_test,  y_test  = faces[test_idx],  labels[test_idx]

print("Train samples:", len(X_train))
print("Test samples:", len(X_test))


Train samples: 1
Test samples: 1


In [10]:
!pip install opencv-contrib-python

In [11]:
import cv2

print("OpenCV version:", cv2.__version__)
print("Has face module:", hasattr(cv2, "face"))

if hasattr(cv2, "face"):
    print("LBPH available:", hasattr(cv2.face, "LBPHFaceRecognizer_create"))
    print(dir(cv2.face))


OpenCV version: 4.12.0
Has face module: True
LBPH available: True
['BIF', 'BIF_create', 'BasicFaceRecognizer', 'EigenFaceRecognizer', 'EigenFaceRecognizer_create', 'FaceRecognizer', 'Facemark', 'FacemarkAAM', 'FacemarkKazemi', 'FacemarkLBF', 'FacemarkTrain', 'FisherFaceRecognizer', 'FisherFaceRecognizer_create', 'LBPHFaceRecognizer', 'LBPHFaceRecognizer_create', 'MACE', 'MACE_create', 'MACE_load', 'PredictCollector', 'StandardCollector', 'StandardCollector_create', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_native', 'createFacemarkAAM', 'createFacemarkKazemi', 'createFacemarkLBF', 'drawFacemarks', 'getFacesHAAR', 'loadDatasetList', 'loadFacePoints', 'loadTrainingData']


In [12]:
model = cv2.face.LBPHFaceRecognizer_create(
    radius=1,
    neighbors=8,
    grid_x=8,
    grid_y=8
)

model.train(X_train, y_train)
print("Model training completed")


Model training completed
